In [7]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from functools import reduce
from django.core.serializers import serialize

In [8]:
from tra.models import *
from tra.test_runs_processing import *

In [9]:
from tra.storage import get_storage_instance
from tra import utils

In [10]:
storage = get_storage_instance()
storage._storage_http_server

'http://127.0.0.1:9000'

In [5]:
# storage.save("2175v2", "http://logs.ute.nsn-rdnet.net/cloud/execution/15632187/")
storage.delete('tra')

In [6]:
pull_passed_testruns_by_testset_filter(13)
# download_latest_passed_logs_by_testset_filter(13)

Starting new HTTPS connection (1): rep-portal.wroclaw.nsn-rdnet.net


['limit=50', 'result__name__pos_neg=passed&test_col__testline_type__pos_neg=CLOUD_5G_I_LO_AP_LO_SANSA_FS_ECPRI_CMWV_TDD&qc_test_instance__test_set__name__pos_neg=5GC002175-QB&qc_test_instance__m_path__pos_neg_empty_str=Root%5CTest_Sets%5CTrunk%5CRAN_L2_SW_KRK_2%5C5GC002175_Spillover_from_SA_Mobility_%285GC000704_5GC000708%29', 'fields=id,airphone,qc_test_set,qc_test_instance,organization,hyperlink_set,test_suite,test_case__name,test_col,test_line,result,env_issue_type,comment,builds,fail_message,start,end']


{'new_runs': [83676557,
  83676556,
  83535989,
  83535984,
  83509530,
  83509527,
  83348643,
  83348642,
  83199575,
  83199574,
  83063901,
  83063898,
  82996159,
  82996152,
  82861609,
  82861606,
  82702208,
  82702202,
  82544965,
  82544963,
  82487450,
  82487441,
  82349580,
  82349572,
  82185542,
  82185536],
 'skipped_runs': [82068772,
  82068770,
  81986025,
  81986021,
  81916994,
  81916990,
  81703399,
  81703394,
  81563846,
  81563834,
  81516677,
  81516675,
  81386503,
  81386499,
  81145773,
  81145771,
  81116523,
  81116521,
  81002740,
  81002736,
  80844662,
  80844659,
  80663887,
  80663881]}

In [11]:
testsetfilter = TestSetFilter.objects.get(id=13)
test_instance = testsetfilter.test_instances.all()
# testruns = test_instance.test_runs.all().filter(result=utils.get_passed_result_instance()).order_by('-end_time')
# for testrun in testruns:
#     print(testrun.end_time)
for ti in test_instance: 
    print(ti.id)

265
355


In [6]:
download_latest_passed_logs_by_testset_filter(13)

In [14]:
from tra.tasks import remove_old_passed_logs_from_log_storage
remove_old_passed_logs_from_log_storage.delay()

<AsyncResult: a700667c-9aa6-41ce-92e1-06607abc3f7a>